<img src="https://i.imgur.com/6U6q5jQ.png"/>

_____

# Analytics on GeodataFrames
 

Let's read the data in:

In [1]:
# data table
import pandas as pd
linkData="https://github.com/SocialAnalytics-StrategicIntelligence/OrganizeExploreAndQuery/raw/main/dataFiles/dengue_ok.pkl"
dengue = pd.read_pickle(linkData)
dengue.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398943 entries, 0 to 398942
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   departamento  398943 non-null  object        
 1   provincia     398943 non-null  object        
 2   distrito      398943 non-null  object        
 3   ano           398943 non-null  int64         
 4   semana        398943 non-null  int64         
 5   sexo          398943 non-null  object        
 6   edad          398943 non-null  int64         
 7   enfermedad    398943 non-null  category      
 8   year          398931 non-null  datetime64[ns]
dtypes: category(1), datetime64[ns](1), int64(3), object(4)
memory usage: 24.7+ MB


In [2]:
dengue.head()

,departamento,provincia,distrito,ano,semana,sexo,edad,enfermedad,year
0,HUANUCO,LEONCIO PRADO,LUYANDO,2000,47,M,9,SIN_SEÑALES,2000-01-01
1,HUANUCO,LEONCIO PRADO,LUYANDO,2000,40,F,18,SIN_SEÑALES,2000-01-01
2,HUANUCO,LEONCIO PRADO,JOSE CRESPO Y CASTILLO,2000,48,F,32,SIN_SEÑALES,2000-01-01
3,HUANUCO,LEONCIO PRADO,JOSE CRESPO Y CASTILLO,2000,37,F,40,SIN_SEÑALES,2000-01-01
4,HUANUCO,LEONCIO PRADO,MARIANO DAMASO BERAUN,2000,42,M,16,SIN_SEÑALES,2000-01-01


In [3]:
dengue.ano.value_counts()

ano
2022    56207
2017    43071
2021    40501
2020    39744
2015    29467
2016    22865
2012    20775
2001    16092
2014    15530
2011    15236
2019    13179
2010    12454
2013    12041
2009    11282
2008    10528
2004     7962
2002     6293
2007     5334
2005     5015
2018     4550
2000     4324
2006     3628
2003     2865
Name: count, dtype: int64

In [4]:
dengue=dengue[dengue.ano>=2012]

In [5]:
dengue.enfermedad.value_counts()

enfermedad
SIN_SEÑALES    249278
ALARMA          47028
GRAVE            1624
Name: count, dtype: int64

In [6]:
dengue_alarma=dengue[dengue.enfermedad!='GRAVE']

dengue_alarma.head()

,departamento,provincia,distrito,ano,semana,sexo,edad,enfermedad,year
101012,ANCASH,CASMA,CASMA,2012,50,F,45,SIN_SEÑALES,2011-01-01
101013,ANCASH,CASMA,CASMA,2012,51,F,3,SIN_SEÑALES,2011-01-01
101014,ANCASH,SANTA,NUEVO CHIMBOTE,2012,24,F,42,SIN_SEÑALES,2011-01-01
101015,ANCASH,SANTA,CHIMBOTE,2012,27,M,9,SIN_SEÑALES,2011-01-01
101016,ANCASH,SANTA,CHIMBOTE,2012,16,F,1,SIN_SEÑALES,2011-01-01


In [8]:


indexList=['ano','departamento','provincia','enfermedad']
aggregator={'enfermedad':[len]}
dengue_provYear=dengue_alarma.groupby(indexList,observed=True).agg(aggregator)
dengue_provYear

enfermedad
                                                      len
ano  departamento provincia        enfermedad            
2012 AMAZONAS     BAGUA            SIN_SEÑALES        290
                                   ALARMA              53
                  CONDORCANQUI     SIN_SEÑALES         20
                  UTCUBAMBA        SIN_SEÑALES        181
                                   ALARMA               9
...                                                   ...
2022 UCAYALI      CORONEL PORTILLO ALARMA             499
                  PADRE ABAD       SIN_SEÑALES        412
                                   ALARMA              87
                  PURUS            SIN_SEÑALES          1
                                   ALARMA               1

[1326 rows x 1 columns]

In [9]:
dengueDraft=dengue_provYear.unstack(3).fillna(0) #leftmost index in rows
dengueDraft

enfermedad       
                                           len       
enfermedad                         SIN_SEÑALES ALARMA
ano  departamento provincia                          
2012 AMAZONAS     BAGUA                  290.0   53.0
                  CONDORCANQUI            20.0    0.0
                  UTCUBAMBA              181.0    9.0
     ANCASH       CASMA                    5.0    0.0
                  SANTA                  895.0   19.0
...                                        ...    ...
2022 TUMBES       ZARUMILLA               89.0    5.0
     UCAYALI      ATALAYA                542.0   92.0
                  CORONEL PORTILLO      2680.0  499.0
                  PADRE ABAD             412.0   87.0
                  PURUS                    1.0    1.0

[751 rows x 2 columns]

In [10]:
dengueDraft['ALARMA_pct']=dengueDraft.iloc[:,1]/(dengueDraft.iloc[:,0] + dengueDraft.iloc[:,1])
dengue_provYear_Alarm_w=dengueDraft['ALARMA_pct'].unstack('ano').fillna(0)
dengue_provYear_Alarm_w

ano                                2012      2013      2014      2015  \
departamento provincia                                                  
AMAZONAS     BAGUA             0.154519  0.015385  0.054545  0.000000   
             BONGARA           0.000000  0.000000  0.000000  0.000000   
             CHACHAPOYAS       0.000000  0.000000  0.000000  0.000000   
             CONDORCANQUI      0.000000  0.000000  0.000000  0.000000   
             UTCUBAMBA         0.047368  0.034091  0.086667  0.086957   
...                                 ...       ...       ...       ...   
TUMBES       ZARUMILLA         0.121739  0.079365  0.214286  0.175325   
UCAYALI      ATALAYA           0.200000  0.875000  0.333333  0.520000   
             CORONEL PORTILLO  0.326643  0.368421  0.371471  0.614286   
             PADRE ABAD        0.374332  0.310924  0.475000  0.264706   
             PURUS             0.000000  0.000000  0.000000  0.000000   

ano                                2016      2017      2018      2019  \
departamento provincia                                                  
AMAZONAS     BAGUA             0.000000  0.000000  0.011111  0.040323   
             BONGARA           0.000000  0.000000  0.000000  0.000000   
             CHACHAPOYAS       0.000000  0.000000  0.000000  1.000000   
             CONDORCANQUI      0.000000  0.000000  0.000000  0.000000   
             UTCUBAMBA         0.121212  0.055556  0.000000  0.000000   
...                                 ...       ...       ...       ...   
TUMBES       ZARUMILLA         0.009091  0.013468  0.111111  0.022727   
UCAYALI      ATALAYA           0.318182  0.255034  0.400000  0.250000   
             CORONEL PORTILLO  0.228498  0.278409  0.190083  0.195876   
             PADRE ABAD        0.412371  0.264706  0.368421  0.400000   
             PURUS             0.000000  0.000000  0.000000  0.000000   

ano                                2020      2021      2022  
departamento provincia                                       
AMAZONAS     BAGUA             0.083333  0.075721  0.077234  
             BONGARA           0.000000  0.000000  0.000000  
             CHACHAPOYAS       0.000000  0.500000  0.125000  
             CONDORCANQUI      0.153285  0.028070  0.012500  
             UTCUBAMBA         0.065268  0.028290  0.047821  
...                                 ...       ...       ...  
TUMBES       ZARUMILLA         0.029762  0.014085  0.053191  
UCAYALI      ATALAYA           0.169591  0.214894  0.145110  
             CORONEL PORTILLO  0.174333  0.137194  0.156968  
             PADRE ABAD        0.194489  0.117647  0.174349  
             PURUS             0.000000  0.000000  0.500000  

[108 rows x 11 columns]

In [11]:
dengue_provYear_Alarm_w.columns

Index([2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022], dtype='int64', name='ano')

In [12]:
dengue_provYear_Alarm_w.columns=['year'+str(x) for x in dengue_provYear_Alarm_w.columns]

In [13]:
dengue_provYear_Alarm_w

year2012  year2013  year2014  year2015  \
departamento provincia                                                  
AMAZONAS     BAGUA             0.154519  0.015385  0.054545  0.000000   
             BONGARA           0.000000  0.000000  0.000000  0.000000   
             CHACHAPOYAS       0.000000  0.000000  0.000000  0.000000   
             CONDORCANQUI      0.000000  0.000000  0.000000  0.000000   
             UTCUBAMBA         0.047368  0.034091  0.086667  0.086957   
...                                 ...       ...       ...       ...   
TUMBES       ZARUMILLA         0.121739  0.079365  0.214286  0.175325   
UCAYALI      ATALAYA           0.200000  0.875000  0.333333  0.520000   
             CORONEL PORTILLO  0.326643  0.368421  0.371471  0.614286   
             PADRE ABAD        0.374332  0.310924  0.475000  0.264706   
             PURUS             0.000000  0.000000  0.000000  0.000000   

                               year2016  year2017  year2018  year2019  \
departamento provincia                                                  
AMAZONAS     BAGUA             0.000000  0.000000  0.011111  0.040323   
             BONGARA           0.000000  0.000000  0.000000  0.000000   
             CHACHAPOYAS       0.000000  0.000000  0.000000  1.000000   
             CONDORCANQUI      0.000000  0.000000  0.000000  0.000000   
             UTCUBAMBA         0.121212  0.055556  0.000000  0.000000   
...                                 ...       ...       ...       ...   
TUMBES       ZARUMILLA         0.009091  0.013468  0.111111  0.022727   
UCAYALI      ATALAYA           0.318182  0.255034  0.400000  0.250000   
             CORONEL PORTILLO  0.228498  0.278409  0.190083  0.195876   
             PADRE ABAD        0.412371  0.264706  0.368421  0.400000   
             PURUS             0.000000  0.000000  0.000000  0.000000   

                               year2020  year2021  year2022  
departamento provincia                                       
AMAZONAS     BAGUA             0.083333  0.075721  0.077234  
             BONGARA           0.000000  0.000000  0.000000  
             CHACHAPOYAS       0.000000  0.500000  0.125000  
             CONDORCANQUI      0.153285  0.028070  0.012500  
             UTCUBAMBA         0.065268  0.028290  0.047821  
...                                 ...       ...       ...  
TUMBES       ZARUMILLA         0.029762  0.014085  0.053191  
UCAYALI      ATALAYA           0.169591  0.214894  0.145110  
             CORONEL PORTILLO  0.174333  0.137194  0.156968  
             PADRE ABAD        0.194489  0.117647  0.174349  
             PURUS             0.000000  0.000000  0.500000  

[108 rows x 11 columns]

In [14]:
dengue_provYear_Alarm_w.reset_index(inplace=True)
dengue_provYear_Alarm_w

,departamento,provincia,year2012,year2013,year2014,year2015,year2016,year2017,year2018,year2019,year2020,year2021,year2022
0,AMAZONAS,BAGUA,0.154519,0.015385,0.054545,0.000000,0.000000,0.000000,0.011111,0.040323,0.083333,0.075721,0.077234
1,AMAZONAS,BONGARA,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,AMAZONAS,CHACHAPOYAS,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.500000,0.125000
3,AMAZONAS,CONDORCANQUI,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.153285,0.028070,0.012500
4,AMAZONAS,UTCUBAMBA,0.047368,0.034091,0.086667,0.086957,0.121212,0.055556,0.000000,0.000000,0.065268,0.028290,0.047821
...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,TUMBES,ZARUMILLA,0.121739,0.079365,0.214286,0.175325,0.009091,0.013468,0.111111,0.022727,0.029762,0.014085,0.053191
104,UCAYALI,ATALAYA,0.200000,0.875000,0.333333,0.520000,0.318182,0.255034,0.400000,0.250000,0.169591,0.214894,0.145110
105,UCAYALI,CORONEL PORTILLO,0.326643,0.368421,0.371471,0.614286,0.228498,0.278409,0.190083,0.195876,0.174333,0.137194,0.156968
106,UCAYALI,PADRE ABAD,0.374332,0.310924,0.475000,0.264706,0.412371,0.264706,0.368421,0.400000,0.194489,0.117647,0.174349


In [ ]:
mapLink='https://github.com/SocialAnalytics-StrategicIntelligence/GeoDF_Analytics/raw/main/maps/DistritalINEI2023.zip'

import geopandas as gpd

datadismap=gpd.read_file(mapLink)

datadismap.info()

In [ ]:
datadismap['location']=['+'.join(x[0]) for x in zip(datadismap.iloc[:,4:7].values)]
datadismap.head(10)

In [ ]:
dengue_provYear_Alarm_w['location']=['+'.join(x[0]) for x in zip(dengue_provYear_Alarm_w.iloc[:,:3].values)]
dengue_provYear_Alarm_w.head()

## Preprocessing

After observing both tables, it would be better if the columns with names have the same capitalization, and no extra blank spaces:

The names from non-english speaking countries may come with some symbols that may cause trouble (', ~). Let's get rid of those:

In [ ]:
import unidecode


byePunctuation=lambda x: unidecode.unidecode(x)
dengue_provYear_Alarm_w['location']=dengue_provYear_Alarm_w['location'].apply(byePunctuation)
datadismap['location']=datadismap['location'].apply(byePunctuation)

Let me see how many district we have:

It would be good making sure no *ghost* appears between words:

In [ ]:
# replacing dashes and multiple spaces by a simple space
dengue_provYear_Alarm_w['location']=dengue_provYear_Alarm_w.location.str.replace("\-|\_|\s+","",regex=True)
datadismap['location']=datadismap.location.str.replace("\-|\_|\s+","",regex=True)

## Merging

We need to merge both tables now. That can happen effectively if both tables have a **key** column: a column (or collection of them) whose values in one table are the same in the other one.

The match need not be exact, but only common values in the *key* are merged.

Let's find out what is NOT matched in each table:

In [ ]:
nomatch_df=set(dengue_provYear_Alarm_w.location)- set(datadismap.location)
nomatch_gdf=set(datadismap.location)-set(dengue_provYear_Alarm_w.location) 

This is what could not be matched:

In [ ]:
len(nomatch_df), len(nomatch_gdf)

The right way to go is using **fuzzy merging** (remember we need  _the fuzz_):

In [ ]:
# pick the closest match from nomatch_gdf for a value in nomatch_df
from thefuzz import process
[(dis,process.extractOne(dis,nomatch_gdf)) for dis in sorted(nomatch_df)]

If you are comfortable, you prepare a _dictionary_ of changes:

In [ ]:
# is this OK?
{dis:process.extractOne(dis,nomatch_gdf)[0] for dis in sorted(nomatch_df)}

In [ ]:
# then:
changesinDF={dis:process.extractOne(dis,nomatch_gdf)[0] for dis in sorted(nomatch_df)}

Now, make the replacements:

In [ ]:
dengue_provYear_Alarm_w.replace({'location': changesinDF}, inplace=True)

In [ ]:
nomatch_df=set(dengue_provYear_Alarm_w.location)- set(datadismap.location)
nomatch_gdf=set(datadismap.location)-set(dengue_provYear_Alarm_w.location) 

[(dis,process.extractOne(dis,nomatch_gdf)) for dis in sorted(nomatch_df)]

Now the merge can happen:

In [ ]:
dengue_provYear_Alarm_map=datadismap.merge(dengue_provYear_Alarm_w, on='location',how='left',indicator='flag')

In [ ]:
# check
dengue_provYear_Alarm_map.info()

In [ ]:
# avoid poblems with fillna()
dengue_provYear_Alarm_map['flag']=dengue_provYear_Alarm_map.flag.astype(str)

We can get rid of some columns:

In [ ]:
bye=['departamento', 'provincia', 'distrito','CCDD','CCPP','CCDD','OBJECTID','ESRI_OID']
dengue_provYear_Alarm_map.drop(columns=bye,inplace=True)

# keeping
dengue_provYear_Alarm_map.head()

In [ ]:
dengue_provYear_Alarm_map.fillna(0,inplace=True)

We can save this gdf:

In [ ]:
import os
dengue_provYear_Alarm_map.to_file(os.path.join('maps',"distritos.gpkg"), layer='distritosDengue', driver="GPKG")

## Exploring one variable

This time, we explore statistically one variable in the map:

In [ ]:
# statistics
dengue_provYear_Alarm_map.year2022.describe()

In [ ]:
dengue_provYear_Alarm_map=dengue_provYear_Alarm_map.to_crs(24892)
dengue_provYear_Alarm_map.crs

In [ ]:
import seaborn as sea

sea.boxplot(dengue_provYear_Alarm_map.year2022, color='yellow',orient='h')

In [ ]:

from sklearn.preprocessing import QuantileTransformer
qt = QuantileTransformer(n_quantiles=100, random_state=0)
qt_result=qt.fit_transform(dengue_provYear_Alarm_map[['year2022']])
sea.boxplot(qt_result, color='yellow',orient='h')

In [ ]:
dengue_provYear_Alarm_map['year_2022_qt']=qt_result

Notice the histogram divides the data in intervals which are the base of the bars. Seaborn uses the [Freedman-Diaconis](https://en.wikipedia.org/wiki/Freedman%E2%80%93Diaconis_rule) formula to compute the bins.

Let's see other possibilities, but please install [**numba**](https://numba.readthedocs.io/en/stable/user/installing.html) before runing the next code; also make sure you have **pysal**, **mapclassify** and **numpy** installed: 

In [ ]:
import mapclassify 
import numpy as np

np.random.seed(12345) # so we all get the same results!
K=5
theVar=dengue_provYear_Alarm_map.year_2022_qt

mb5 = mapclassify.MaximumBreaks(theVar, k=K)

In [ ]:
# group label
mb5.yb

In [ ]:
# labels and counts
np.unique(mb5.yb,return_counts=True)
          

In [ ]:
# ht.yb into a pandas Series

pd.Series(mb5.yb).value_counts()

In [ ]:
# these are the cuts, but it is not including the min value
mb5.bins

Based on the previous information, let me prepare a histogram:

In [ ]:
# completing the bins
MB_bins=list(mb5.bins)
MB_bins.insert(0,dengue_provYear_Alarm_map.year_2022_qt.min())

In [ ]:
sea.displot(dengue_provYear_Alarm_map.year_2022_qt, bins=MB_bins,color='yellow')

In [ ]:
dengue_provYear_Alarm_map['year_2022_qt'] = mb5.yb 

Let me plot one of them:

In [ ]:
import matplotlib.pyplot as plt

f, ax = plt.subplots(1, figsize=(9, 9))
dengue_provYear_Alarm_map.plot(column='year_2022_qt', 
        cmap='viridis', 
        categorical=True,
        edgecolor='white', 
        linewidth=0., 
        alpha=0.75, 
        legend=True,
        legend_kwds=dict(loc=2),
        ax=ax
       )

ax.set_axis_off()

## Spatial Correlation

### Neighboorhood

We can compute the neighborhood in a map using different algorithms:

In [ ]:
from libpysal.weights import Queen, Rook, KNN

# rook
w_rook = Rook.from_dataframe(dengue_dist_projected) 

In [ ]:
# rook
w_queen = Queen.from_dataframe(dengue_dist_projected)

In [ ]:
# k nearest neighbors
w_knn = KNN.from_dataframe(dengue_dist_projected, k=8)

Let's understand the differences:

In [ ]:
# first one
dengue_dist_projected.head(1)

In [ ]:
# amount neighbors of that district
w_rook.neighbors[0] 

In [ ]:
# see
base=dengue_dist_projected[dengue_dist_projected.DISTRITO=="CHACHAPOYAS"].plot()
dengue_dist_projected.iloc[w_rook.neighbors[0] ,].plot(ax=base,facecolor="yellow",edgecolor='k')
dengue_dist_projected.head(1).plot(ax=base,facecolor="red")

Let's do the same:

In [ ]:
w_queen.neighbors[0]

In [ ]:
base=dengue_dist_projected[dengue_dist_projected.DISTRITO=="CHACHAPOYAS"].plot()
dengue_dist_projected.iloc[w_queen.neighbors[0] ,].plot(ax=base,facecolor="yellow",edgecolor='k')
dengue_dist_projected.head(1).plot(ax=base,facecolor="red")

In [ ]:
w_knn.neighbors[0]

In [ ]:
base=dengue_dist_projected[dengue_dist_projected.DISTRITO=="CHACHAPOYAS"].plot()
dengue_dist_projected.iloc[w_knn.neighbors[0] ,].plot(ax=base,facecolor="yellow",edgecolor='k')
dengue_dist_projected.head(1).plot(ax=base,facecolor="red")

Let me pay attention to the queen results:

In [ ]:
# all the neighbors by row
w_queen.neighbors

In [ ]:
# the matrix of neighboorhood:

pd.DataFrame(*w_queen.full()).astype(int) # 1 means both are neighbors

In [ ]:
# count of nonzeros
w_queen.nonzero

In [ ]:
# pct of neighboorhood (density)
w_queen.pct_nonzero

In [ ]:
# a district with NO neighbor?
w_queen.islands

### Exercise 5

<div class="alert-success">
    
1. Compute the three neighboohoods shown above for you data.
    
2. Select one polygon, and plot it with its neighbors as above.
    
</div>

## Spatial correlation

We need the neighboorhood matrix (the weight matrix) to compute spatial correlation: if the variable value is correlated with the values of its neighbors - which proves a spatial effect.

In [ ]:
# needed for spatial correlation
w_queen.transform = 'R'

Spatial correlation is measured by the Moran's I statistic:

In [ ]:
from esda.moran import Moran

moranDENGUE = Moran(dengue_dist_projected['alarm_km2_2022_qt'], w_queen)
moranDENGUE.I,moranDENGUE.p_sim

A significant Moran's I suggest spatial correlation. Let's see the spatial scatter plot

In [ ]:
from splot.esda import moran_scatterplot

fig, ax = moran_scatterplot(moranDENGUE)#, aspect_equal=True)
ax.set_xlabel('dengue')
ax.set_ylabel('SpatialLag_dengue')
plt.show()

### Exercise 6

<div class="alert-success">
    
1. Compute the Moran's coefficient for **all** your numeric variables.
    
2. Make a scatter plot for each variable.
    
</div>

### Local Spatial Correlation

We can compute a LISA (local Moran) for each case. That will help us find spatial clusters (spots) and spatial outliers:

* A **hotSpot** is a polygon whose value in the variable is high AND is surrounded with polygons with also high values.

* A **coldSpot** is a polygon whose value in the variable is low AND is surrounded with polygons with also low values.

* A **coldOutlier** is a polygon whose value in the variable is low BUT is surrounded with polygons with  high values.

* A **hotOutlier** is a polygon whose value in the variable is high BUT is surrounded with polygons with  low values.

It is also possible that no significant correlation is detected. Let's see those values:

In [ ]:
# The scatterplot with local info
from esda.moran import Moran_Local

# calculate Moran_Local and plot
lisaDENGUE = Moran_Local(y=dengue_dist_projected['alarm_km2_2022_qt'], w=w_queen,seed=2022)
fig, ax = moran_scatterplot(lisaDENGUE,p=0.05)
ax.set_xlabel('dengue')
ax.set_ylabel('SpatialLag_dengue')
plt.show()

In [ ]:
# the map with the spots and outliers

from splot.esda import lisa_cluster
f, ax = plt.subplots(1, figsize=(12, 12))
plt.title('Spots and Outliers')
fig = lisa_cluster(lisaDENGUE, 
                   dengue_dist_projected,ax=ax,
                   legend_kwds={'loc': 'center left', 
                                'bbox_to_anchor': (0.7, 0.6)})
plt.show()

Let me add that data to my gdf:

In [ ]:
# quadrant
lisaDENGUE.q

In [ ]:
# significance
lisaDENGUE.p_sim

In [ ]:
# quadrant: 1 HH,  2 LH,  3 LL,  4 HL
pd.Series(lisaDENGUE.q).value_counts()

The info in **lisaIDH.q** can not be used right away, we need to add if the local spatial correlation is significant:

In [ ]:
dengue_dist_projected['DENGUE_quadrant']=[l if p <0.05 else 0 for l,p in zip(lisaDENGUE.q,lisaDENGUE.p_sim)  ]
dengue_dist_projected['DENGUE_quadrant'].value_counts()

Now, we recode:

In [ ]:
labels = [ '0 no_sig', '1 hotSpot', '2 coldOutlier', '3 coldSpot', '4 hotOutlier']

dengue_dist_projected['DENGUE_quadrant_names']=[labels[i] for i in dengue_dist_projected['DENGUE_quadrant']]

dengue_dist_projected['DENGUE_quadrant_names'].value_counts()
                                  

Let's replot:

In [ ]:
from matplotlib import colors
myColMap = colors.ListedColormap([ 'ghostwhite', 'red', 'green', 'black','orange'])



# Set up figure and ax
f, ax = plt.subplots(1, figsize=(12,12))
# Plot unique values choropleth including
# a legend and with no boundary lines

plt.title('Spots and Outliers')

dengue_dist_projected.plot(column='DENGUE_quadrant_names', 
                categorical=True,
                cmap=myColMap,
                linewidth=0.1, 
                edgecolor='white',
                legend=True,
                legend_kwds={'loc': 'center left', 
                             'bbox_to_anchor': (0.7, 0.6)},
                ax=ax)
# Remove axis
ax.set_axis_off()
# Display the map
plt.show()

In [ ]:
dengue_dist_projected.explore("DENGUE_quadrant_names", categorical=True,tooltip='location')

### Exercise 7

<div class="alert-success">
    
1. Compute the Local Moran for the variables in your data that have significant spatial correlation.
    
2. Create a new column for each of those variables, with a label ('0 no_sig', '1 hotSpot', '2 coldOutlier', '3 coldSpot', '4 hotOutlier').

3. Prepare a map for each of the variables analyzed, showing the spots and outliers.
    
</div>